In [1]:
# Imports
import pandas as pd
import json
import re

In [23]:
# Read file
input_filename = 'df_results_mistral_multilabel.csv'
df = pd.read_csv(input_filename)

In [24]:
df["model_output"][165]

' [{"text": "I\'ll correct you lol because I don\'t want it to look spiteful and NPHC\'s demeanor towards Greek life is a little different... TL;DR: rushed and rejected from one sorority, didn\'t like how the chapter handles themselves as a whole, should I rush another sorority that may align with my views better even if they are \'rivals\'.", "class": "hasty generalization"}]\n\nExplanation: The author is making a hasty generalization about NPHC\'s demeanor towards Greek life based on her experience with one sorority. It is not fair to judge an entire organization or group based on the actions of a single chapter.'

In [4]:
def extract_json_string(model_output):
    """
    Extracts valid JSON strings from the text within square brackets [].
    Only includes JSON objects that have a non-empty 'class' field.
    """
    try:
        # Find all substrings within square brackets
        json_strings = re.findall(r'\[.*?\]', model_output, re.DOTALL)
        
        valid_json_objects = []
        
        for json_str in json_strings:
            try:
                # Clean the JSON string to ensure it's in valid JSON format
                json_str = json_str.replace('\\"', '\"').replace("\\'", "'")
                json_objects = json.loads(json_str)
                
                for obj in json_objects:
                    if obj.get("class"):
                        valid_json_objects.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e} in string: {json_str}")
                continue
        
        # Convert the list of valid JSON objects back to a JSON string
        if valid_json_objects:
            return json.dumps(valid_json_objects)
        
        return "[]"
    except Exception as e:
        print(f"Error extracting JSON: {e}")
        return "[]"

df['json_string'] = df['model_output'].apply(extract_json_string)

Error decoding JSON: Expecting ',' delimiter: line 1 column 472 (char 471) in string: [{"text": "These are unidentified people, and after the bomb goes off, they're blown to bits because they are suicidal individuals who think they're going to go to paradise if they perpetrate such an act and lose their life in doing it.", "class": "Ad hominem"}, {"text": "We're busy trying to find the centers where these operations stem from, and retaliation will be taken.", "class": "Appeal to fear"}, {"text": "But we're not going to simply kill some people to say,"Oh, look, we got even.''", "class": "Appeal to pity"}, {"text": "Every time I bring my umbrella with me, it rains. Clearly, if I leave it at home, there will be sunshine!", "class": "False causality"}, {"text": "I water the garden every day, which is why it grows so well.", "class": "Causal oversimplification"}, {"text": "My favorite actor, who starred in that movie about a virus that turns people into zombies, said in an interview that ge

In [21]:
df["model_output"][73]

'[{"text": "I worked in an engineering factory for years and was never supplied a back belt so I now have a bad back and the only way I can get any sleep is to take 4 pain killers before bed or smoke one joint so I\'ll stick to what\'s natural rather than use so many painkillers my liver will be useless in a few years so cannabis for the win with me", "class": "appeal to worse problems"}]\n\nNote: The text contains an appeal to worse problems, where the author is saying that the alternative (using painkillers) is worse than the option they are advocating for (using cannabis), and therefore, they will choose the latter.'

In [25]:
df["json_string"].head(27)

0     [{"text": "I keep seeing people asking for hig...
1     [{"text": "Two of my best friends are really i...
2     [{"text": "Someone once told me they have an '...
3     [{"text": "Joe Biden will lose to Trump if he ...
4     [{"text": "America is the best place to live, ...
5     [{"text": "Identify if this text contains the ...
6     [{"text": "Delores is a big supporter for equa...
7     [{"text": "Four out of five dentists recommend...
8     [{"text": "police need to interact with people...
9     [{"text": "The last Democrat winner of the New...
10    [{"text": "Men score better on math than women...
11    [{"text": "Identify if this text contains the ...
12    [{"text": "Carbon dioxide hurts nobody's healt...
13                                                   []
14    [{"text": "Iran has moved forward with its nuc...
15    [{"text": "I wonder how is New York planning o...
16    [{"text": "Jeff surveys high school students i...
17    [{"text": "The Freehold Raceway Mall is th

In [7]:
df.to_csv(input_filename, index=False)